<style>
/* Custom styling for Jupyter notebook */
:root {
    --main-bg-color: #f8f9fa;
    --text-color: #212529;
    --link-color: #3498db;
    --code-bg: #f8f9fa;
    --success-color: #2ecc71;
    --error-color: #e74c3c;
    --border-color: #dee2e6;
    --heading-color: #2c3e50;
    --notebook-width: 800px;  /* Control notebook width here */
}

body {
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
    line-height: 1.6;
    max-width: var(--notebook-width);
    margin: 0 auto;
    padding: 2rem;
    background-color: var(--main-bg-color);
    color: var(--text-color);
}

/* Force elements to respect max width */
.jp-OutputArea-output {
    max-width: var(--notebook-width);
    margin: 0 auto;
}

/* Table responsiveness */
div.jp-RenderedHTMLCommon table {
    max-width: var(--notebook-width); 
    overflow-x: auto;
    display: block;
}

/* Code cells width control */
.jp-Cell-inputWrapper, .jp-Cell-outputWrapper {
    max-width: var(--notebook-width);
}

/* Image width control */
img {
    max-width: 100%;
    margin: 0 auto;
    display: block;
}

/* Rest of the styles remain the same */
.notebook {
    background: white;
    padding: 2rem;
    border-radius: 8px;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
}

div.cell {
    margin-bottom: 1.5rem;
    border: none;
}

div.text_cell_render {
    padding: 1rem 0;
    line-height: 1.8;
}

h1, h2, h3, h4 {
    color: var(--heading-color);
    margin-top: 2rem;
    margin-bottom: 1rem;
    font-weight: 600;
}

h1 { font-size: 2.5rem; }
h2 { font-size: 2rem; }
h3 { font-size: 1.5rem; }

div.input_area {
    background-color: var(--code-bg);
    border-radius: 6px;
    padding: 1rem;
    margin: 0.5rem 0;
}

div.output_area {
    margin: 0.5rem 0;
    padding: 1rem;
    background: white;
    border-radius: 6px;
}

div.output_text pre {
    border-left: 4px solid var(--success-color);
    margin: 0.5rem 0;
    padding-left: 1rem;
}

div.output_stderr pre {
    background-color: #fff5f5;
    border-left: 4px solid var(--error-color);
    color: var(--error-color);
    padding-left: 1rem;
}

code {
    font-family: 'Fira Code', 'JetBrains Mono', monospace;
    font-size: 0.9em;
}

pre {
    background-color: var(--code-bg);
    padding: 1rem;
    border-radius: 6px;
    overflow-x: auto;
}

a {
    color: var(--link-color);
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}

img {
    max-width: 100%;
    height: auto;
    border-radius: 4px;
    margin: 1rem 0;
}

table {
    width: 100%;
    margin: 1rem 0;
    border-collapse: collapse;
}

th, td {
    padding: 0.75rem;
    border: 1px solid var(--border-color);
}

th {
    background-color: var(--code-bg);
    font-weight: 600;
}

/* Syntax highlighting */
.highlight .k { color: #8959a8; } /* Keyword */
.highlight .n { color: #4d4d4c; } /* Name */
.highlight .s { color: #718c00; } /* String */
.highlight .c { color: #8e908c; font-style: italic; } /* Comment */

/* Dark mode support */
@media (prefers-color-scheme: dark) {
    :root {
        --main-bg-color: #1a1a1a;
        --text-color: #e0e0e0;
        --link-color: #61afef;
        --code-bg: #282c34;
        --success-color: #98c379;
        --error-color: #e06c75;
        --border-color: #3e4451;
        --heading-color: #61afef;
    }

    body {
        background-color: var(--main-bg-color);
        color: var(--text-color);
    }

    .notebook {
        background: #282c34;
    }

    div.output_area {
        background: #282c34;
    }

    .highlight .n { color: #abb2bf; }
}
</style>

# Local LLM Tutorial: Streaming Text Generation

---

## Table of Contents
1. [Introduction](#introduction)
2. [Understanding the Pipeline](#understanding-the-pipeline)
3. [Model Download](#model-download)
4. [Model Loading](#model-loading)
5. [Setting Up Streaming](#streaming)
6. [Generation Parameters](#parameters)
7. [KV Caching Optimization](#kv-caching)
8. [Running the Generation](#running)
9. [Try Different Queries](#queries)

---

## Introduction

Language models have become increasingly powerful, but their practical deployment often requires
efficient handling of text generation, especially in interactive applications. Streaming generation
is crucial for creating responsive user experiences, as it allows for real-time display of
generated text rather than waiting for the entire output.

### What We'll Cover:

1. Set up a local language model using the Hugging Face pipeline
2. Implement streaming text generation
3. Configure generation parameters for optimal output
4. Understand the underlying components and their interactions

> **Important Note**: Some models (such as Mistral models) on the Hugging Face Hub require accepting terms of use (gated models).
> 
> Requirements:
> - Log in to your Hugging Face account (or create one)
> - Accept the terms on the model's page
> - Set your Hugging Face API token as the `HF_TOKEN` variable in a `.env` file

In [ ]:
from threading import Thread

import torch
import dotenv
from transformers import pipeline
from transformers import TextIteratorStreamer

dotenv.load_dotenv('.env')

# Choose a Model
model_id = 'deepcogito/cogito-v1-preview-llama-3B'
#model_id = 'HuggingFaceTB/SmolLM2-1.7B-Instruct'
#model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

## Understanding the Pipeline

The Hugging Face pipeline is a high-level abstraction that combines several components:

```mermaid
graph LR
    Input["Input Text"] --> Tokenizer
    Tokenizer --> Preprocessor
    Preprocessor --> Model
    Model --> Postprocessor
    Postprocessor --> Output["Output Text"]
    Output -.->|Forward Pass| Model
    
    style Input fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    style Tokenizer fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    style Preprocessor fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    style Model fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    style Postprocessor fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    style Output fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    
    linkStyle 5 stroke:#333,stroke-width:1px,stroke-dasharray:5 5
```

### Core Components

#### Model
- Core language model (causal LM)
- Transformer architecture
- Model weights and definition
- Forward pass handling

#### Tokenizer
- Text to token conversion
- Subword tokenization
- Special token handling
- Vocabulary management

#### Preprocessor
- Input preparation
- Chat template application
- Padding and truncation
- Attention mask creation

#### Postprocessor
- Output processing
- Token ID decoding
- Special token removal
- Output formatting

---

## Model Download

Available models for this tutorial:

| Model             | Size | Description               |
|:-----------------|:-----:|:--------------------------|
| cogito-v1-preview | 3B   | Fast, lightweight model  |
| SmolLM2          | 1.7B | Efficient instruction    |
| Mistral          | 7B   | Powerful, state-of-the-art|

In [ ]:
# Load the model with optimized settings
pipe = pipeline(
    'text-generation',
    model=model_id,
    model_kwargs={'torch_dtype': torch.bfloat16},
    device_map='auto'
)

## Setting Up Streaming

The streaming process involves three key components working together:

```mermaid
graph LR
    Main["Main Thread"] -->|Sends Input| Queue["Token Queue"]
    Worker["Worker Thread"] -->|Generates| Queue
    Queue -->|Buffers| Output["Output Text"]
    
    style Main fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    style Worker fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    style Queue fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    style Output fill:#f0f0ff,stroke:#9999ff,stroke-width:2px
    
    linkStyle 0 stroke:#333,stroke-width:1px
    linkStyle 1 stroke:#333,stroke-width:1px
    linkStyle 2 stroke:#333,stroke-width:1px
```

During streaming generation:
- The main thread sends inputs to the model and monitors the token queue
- The worker thread continuously generates tokens and adds them to the queue
- Tokens are consumed from the queue and displayed in real-time

---

## Generation Parameters

### Precision Controls
| Parameter        | Value | Purpose          |
|:----------------|:-----:|:-----------------|
| `max_new_tokens`| 2048  | Output length    |
| `do_sample`     | True  | Enable sampling  |

### Creative Controls
| Parameter     | Value | Purpose          |
|:-------------|:-----:|:-----------------|
| `temperature`| 0.7   | Randomness       |
| `top_p`      | 0.95  | Diversity        |

### KV Cache Controls
| Parameter             | Value    | Purpose                      |
|:---------------------|:--------:|:-----------------------------|
| `use_cache`          | True     | Enable KV caching            |
| `cache_implementation`| dynamic  | Caching strategy             |

In [ ]:
# Configure the streamer
streamer = TextIteratorStreamer(
    pipe.tokenizer, 
    skip_prompt=True, 
    skip_special_tokens=False
)

## KV Caching Optimization

KV (Key-Value) caching is a crucial optimization technique for efficient text generation with transformer models.

### What is KV Caching?

During autoregressive generation, a model needs to process all previously generated tokens to produce the next token. KV caching stores the intermediate attention key (K) and value (V) states from previous generation steps, avoiding redundant recalculations.

```
Without KV Cache:
[token₁] → compute KV → output token₁
[token₁, token₂] → compute KV → output token₂
[token₁, token₂, token₃] → compute KV → output token₃
...

With KV Cache:
[token₁] → compute KV → cache KV → output token₁
[token₂] → use cached KV + compute new KV → cache KV → output token₂
[token₃] → use cached KV + compute new KV → cache KV → output token₃
...
```

### Caching Implementations

#### Static Caching
- Pre-allocates memory for the entire sequence length
- More memory usage but consistent performance
- Best for fixed-length generations

#### Dynamic Caching
- Allocates memory as needed during generation
- More efficient for variable-length outputs
- Reduces initial memory footprint

#### Offloaded Caching
- Stores KV cache in CPU memory instead of GPU
- Enables generation with larger context lengths
- Trades speed for memory efficiency

### Performance Benefits

| Aspect        | Improvement     | Impact                              |
|:--------------|:----------------:|:------------------------------------|
| Speed         | 2-4× faster     | Reduced time for token generation   |
| Computation   | O(n) → O(1)*    | Constant time for each new token    |
| Memory        | Higher usage    | Requires more GPU VRAM              |
| Scalability   | Limited by VRAM | Max context depends on memory       |

*For attention computation per token

In [ ]:
# Set up the generation parameters
messages = [
    {'role': 'system', 'content': 'You are a helpful AI assistant.'},
    {'role': 'user', 'content': 'What are black holes?'}
]

generation_kwargs = {
    # Input configuration
    'text_inputs': messages,
    'streamer': streamer,

    # KV Cache settings
    'use_cache': True,
    'cache_implementation': 'dynamic',  # 'dynamic', 'static' or offloaded
    
    # Generation settings
    'max_new_tokens': 2048,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.95
}

# Start generation in a thread
thread = Thread(target=pipe, kwargs=generation_kwargs)
thread.start()

# Print tokens as they're generated
for text in streamer:
    print(text, end='', flush=True)

## Running the Generation

When you run the code above, you'll see the model generating text token by token in real-time.
The streaming process allows for immediate feedback and a more interactive experience.

### What's Happening Behind the Scenes?

1. The main thread sets up the generation parameters and creates a streamer
2. A worker thread is spawned to handle the actual text generation
3. As tokens are generated, they're sent to the streamer's queue
4. The main thread reads from the queue and prints tokens in real-time

### Performance Considerations

- Streaming has minimal overhead compared to regular generation
- Memory usage remains constant regardless of output length
- CPU usage is distributed between threads
- GPU utilization remains high during generation

---

## Try Different Queries

Here are some example queries to try:

### Knowledge-Based
```python
queries = [
    "Explain quantum entanglement",
    "What is the theory of relativity?",
    "How do neural networks work?"
]
```

### Creative
```python
queries = [
    "Write a short story about time travel",
    "Compose a haiku about autumn",
    "Create a recipe for a unique sandwich"
]
```

### Technical
```python
queries = [
    "Explain how garbage collection works in Python",
    "What are the SOLID principles in software design?",
    "Compare different sorting algorithms"
]
```

### Analytical
```python
queries = [
    "Analyze the impact of social media on society",
    "Compare renewable energy sources",
    "Discuss the future of artificial intelligence"
]
```

To try a different query, simply modify the `text_inputs` in the `generation_kwargs`:

```python
generation_kwargs['text_inputs'] = [
    {'role': 'system', 'content': 'You are a helpful AI assistant.'},
    {'role': 'user', 'content': 'Your new query here'}
]
```

### Experiment with Parameters

Try adjusting these parameters to see how they affect the output:

| Parameter     | Range  | Effect                    |
|:-------------|:------:|:--------------------------|
| Temperature  | 0-1    | Higher = more random      |
| Top-p        | 0-1    | Higher = more diverse     |
| Max tokens   | 1-2048 | Controls response length  |

Example:
```python
generation_kwargs.update({
    'temperature': 0.9,  # More creative
    'top_p': 0.99,      # More diverse
    'max_new_tokens': 1024  # Shorter response
})
```